# <font color='264CC7'> notion_uploader </font>

En construcción.

Los paquetes necesarios son:

In [1]:
# !pip install pymupdf 
# !pip install openai
# !pip install notion-client
# !pip install pyTelegramBotAPI

In [ ]:
# Paquetes necesarios
import os
import base64
from datetime import datetime
from openai import OpenAI 
import json
from notion_client import Client
import telebot

In [ ]:
# Asignar clave API de OpenAI
from keys import OPENAI_API_KEY, NOTION_TOKEN, NOTION_DATABASE_ID, TELEGRAM_TOKEN, USUARIO_AUTORIZADO

client = OpenAI(api_key=OPENAI_API_KEY)
notion = Client(auth=NOTION_TOKEN)
bot = telebot.TeleBot(TELEGRAM_TOKEN)

---
## <font color='264CC7'> Lectura de image y extracción de datos </font>


In [ ]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
    
# 🤖 Función: Analizar con GPT
def analizar_imagen_cerveza(ruta_imagen):
    
    base64_image = encode_image(ruta_imagen)
    
    # URL de la imagen en formato base64
    image_url = f"data:image/jpeg;base64,{base64_image}"
    
    # Instrucciones para el modelo (System Prompt)
    system_prompt = """
    Eres un asistente experto en análisis de menús de cervezas.
    Analiza la imagen proporcionada y extrae la siguiente información de cada cerveza que encuentres:
    - Nombre de la cerveza
    - Grados de alcohol (ABV) (sin el símbolo de porcentaje)
    - IBU (Unidades Internacionales de Amargor)

    Si la información de ABV o IBU no se encuentra visible en la imagen para alguna cerveza, asigna el valor 0 en su lugar.
    Debes devolver estrictamente un objeto JSON con la estructura solicitada, que contenga un array de cervezas.
    """

    # Llamada única al modelo (visión y extracción JSON)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                {
                    "type": "text", 
                    "text": "Extrae la información de todas las cervezas visibles en esta imagen y devuélvela como un objeto JSON con el array 'cervezas' y las claves 'nombre', 'abv' e 'ibu' para cada cerveza.",
                },
                {
                    "type": "image_url",
                    "image_url": {"url": image_url},
                },
            ]},
        ],
        # Configuración para forzar la respuesta en formato JSON
        response_format={"type": "json_object"}, 
        temperature=0.0
    )
    
    # La respuesta es un string JSON que necesita ser cargado en Python
    json_str = response.choices[0].message.content
    return json.loads(json_str)

In [ ]:
# ruta = "imagenes/test01.jpg"
# json_data = analizar_imagen_cerveza(ruta)
# # Cantidad de caracteres extraídos
# print(len(json_data))
# print(json_data)

1
{'cervezas': [{'nombre': 'Amatsiana', 'abv': '6', 'ibu': '10'}, {'nombre': 'Catequilla', 'abv': '4.7', 'ibu': '20'}, {'nombre': 'Sierra Negra', 'abv': '4.8', 'ibu': '20'}, {'nombre': 'Concordia', 'abv': '5.3', 'ibu': '25'}, {'nombre': 'Apachita', 'abv': '5.9', 'ibu': '20'}, {'nombre': 'Punta Polinar', 'abv': '7.3', 'ibu': '55'}]}


---
## <font color='264CC7'> Envío a Notion </font>

In [6]:
# json_data = {'cervezas': [{'nombre': 'Antisana', 'abv': '6', 'ibu': '10'}, {'nombre': 'Catequilla', 'abv': '4.7', 'ibu': '20'}, {'nombre': 'Sierra Negra', 'abv': '4.8', 'ibu': '20'}, {'nombre': 'Concordia', 'abv': '5.3', 'ibu': '25'}, {'nombre': 'Apachita', 'abv': '5.9', 'ibu': '20'}, {'nombre': 'Puña Palmar', 'abv': '5.9', 'ibu': '32'}, {'nombre': 'Reinventador', 'abv': '7.3', 'ibu': '55'}]}

In [7]:
# json_data["cervezas"]

In [8]:
# 📤 Función: Enviar a Notion
def enviar_cervezas_a_notion(datos, lugar):
    # Fecha actual sin hora ISO 8601
    fecha_actual = datetime.now().strftime("%Y-%m-%d")

    for cerveza in datos["cervezas"]:
        nombre = cerveza.get("nombre", "Sin nombre")
        abv = float(cerveza.get("abv", 0))
        ibu = int(cerveza.get("ibu", 0))

        propiedades = {
            "Nombre": {
                "title": [{"text": {"content": nombre}}]
            },
            "Grados": {
                "number": abv
            },
            "IBU": {
                "number": ibu
            },
            "Lugar": {
                "multi_select": [{"name": lugar}]
            },
            "Fecha": {
                "date": {
                    "start": fecha_actual,
                }
            },
        }

        notion.pages.create(
            parent={"database_id": NOTION_DATABASE_ID},
            properties=propiedades
        )

In [9]:
# enviar_cervezas_a_notion(json_data,"Latitud Cero")

---
## <font color='264CC7'> Bot de Telegram </font>

In [ ]:
usuarios = {}

# Restricción de acceso
def es_usuario_autorizado(message):
    return message.from_user.id == int(USUARIO_AUTORIZADO)

def acceso_restringido(func):
    def wrapper(message):
        if not es_usuario_autorizado(message):
            bot.reply_to(message, "⛔ No estás autorizado para usar este bot.")
            return
        return func(message)
    return wrapper

# Comando /start
@bot.message_handler(commands=['start'])
@acceso_restringido
def start(message):
    bot.send_message(message.chat.id, "🍻 ¿Dónde estás probando las cervezas?")
    usuarios[message.chat.id] = {"estado": "esperando_lugar"}

# Capturar el lugar
@bot.message_handler(func=lambda msg: msg.chat.id in usuarios and usuarios[msg.chat.id]["estado"] == "esperando_lugar")
@acceso_restringido
def recibir_lugar(message):
    usuarios[message.chat.id]["lugar"] = message.text.strip()
    usuarios[message.chat.id]["estado"] = "esperando_foto"
    bot.send_message(message.chat.id, "📷 Ahora envíame una foto de la carta o etiquetas de cervezas.")

# Capturar imagen
@bot.message_handler(content_types=['photo'])
@acceso_restringido
def recibir_foto(message):
    estado = usuarios.get(message.chat.id, {})
    if estado.get("estado") != "esperando_foto":
        bot.reply_to(message, "Primero dime dónde estás (usa /start).")
        return

    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    ruta_img = f"imagenes/{file_id}.jpg"
    os.makedirs("imagenes", exist_ok=True)
    with open(ruta_img, 'wb') as f:
        f.write(downloaded_file)

    bot.send_message(message.chat.id, "🔍 Procesando la imagen...")

    try:
        datos = analizar_imagen_cerveza(ruta_img) 
        lugar = usuarios[message.chat.id]["lugar"]
        enviar_cervezas_a_notion(datos, lugar)  
        bot.send_message(message.chat.id, "✅ Cervezas registradas exitosamente en Notion.")
        bot.send_message(message.chat.id, "Califica las cervezas aquí: https://www.notion.so/a5e415e423764b9cbe76ff6834f09e1d?v=ec1be78074934557979a77428b38abab")
    except Exception as e:
        bot.send_message(message.chat.id, f"⚠️ Error al procesar la imagen: {str(e)}")

    usuarios.pop(message.chat.id, None)

# Fallback
@bot.message_handler(func=lambda message: True)
def fallback(message):
    if not es_usuario_autorizado(message):
        return
    bot.reply_to(message, "Por favor, inicia con /start para registrar una cerveza.")

# Ejecutar bot
bot.polling()
